In [1]:
# mount google drive and export your work
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install the required kafka packages

In [2]:
!pip install kafka-python
!pip install xmltodict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 14.8 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
import pandas as pd

In [24]:
# prompt: untar the tgz file

!tar -xvzf filename.tg

tar (child): filename.tg: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [4]:
!curl -sSOL https://downloads.apache.org/kafka/3.7.1/kafka_2.12-3.7.1.tgz
!tar -xvzf kafka_2.12-3.7.1.tgz

kafka_2.12-3.7.1/
kafka_2.12-3.7.1/LICENSE
kafka_2.12-3.7.1/NOTICE
kafka_2.12-3.7.1/bin/
kafka_2.12-3.7.1/bin/kafka-delete-records.sh
kafka_2.12-3.7.1/bin/trogdor.sh
kafka_2.12-3.7.1/bin/kafka-jmx.sh
kafka_2.12-3.7.1/bin/connect-mirror-maker.sh
kafka_2.12-3.7.1/bin/kafka-console-consumer.sh
kafka_2.12-3.7.1/bin/kafka-consumer-perf-test.sh
kafka_2.12-3.7.1/bin/kafka-log-dirs.sh
kafka_2.12-3.7.1/bin/kafka-metadata-quorum.sh
kafka_2.12-3.7.1/bin/zookeeper-server-stop.sh
kafka_2.12-3.7.1/bin/kafka-verifiable-consumer.sh
kafka_2.12-3.7.1/bin/kafka-features.sh
kafka_2.12-3.7.1/bin/kafka-acls.sh
kafka_2.12-3.7.1/bin/zookeeper-server-start.sh
kafka_2.12-3.7.1/bin/kafka-server-stop.sh
kafka_2.12-3.7.1/bin/kafka-configs.sh
kafka_2.12-3.7.1/bin/kafka-reassign-partitions.sh
kafka_2.12-3.7.1/bin/connect-plugin-path.sh
kafka_2.12-3.7.1/bin/kafka-leader-election.sh
kafka_2.12-3.7.1/bin/kafka-producer-perf-test.sh
kafka_2.12-3.7.1/bin/kafka-transactions.sh
kafka_2.12-3.7.1/bin/kafka-topics.sh
kafka_2.

In [5]:
!./kafka_2.12-3.7.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-3.7.1/config/zookeeper.properties
!./kafka_2.12-3.7.1/bin/kafka-server-start.sh -daemon ./kafka_2.12-3.7.1/config/server.properties
!echo "Give the processes 10 seconds to start before proceeding."
!sleep 10

Give the processes 10 seconds to start before proceeding.


In [6]:
!ps -ef | grep java

root        1357       1 20 17:16 ?        00:00:02 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:
root        1764       1 66 17:16 ?        00:00:07 java -Xmx1G -Xms1G -server -XX:+UseG1GC -XX:MaxG
root        1897     238  0 17:16 ?        00:00:00 /bin/bash -c ps -ef | grep java
root        1899    1897  0 17:16 ?        00:00:00 grep java


# Task I - Create a new Kafka topic and publish data into it as a Producer.

## Creating Topic


In [40]:
!./kafka_2.12-3.7.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic blue-line

Error while executing topic command : Topic 'blue-line' already exists.
[2024-10-26 17:31:54,416] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'blue-line' already exists.
 (org.apache.kafka.tools.TopicCommand)


Describe the topic for details on the configuration


In [41]:
!./kafka_2.12-3.7.1/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic blue-line

Topic: blue-line	TopicId: ZhZEdWY3SneTaOsfWH6B6Q	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: blue-line	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


In [16]:
import requests

In [42]:
token_dict = {'token':userdata.get('open_weather')}
token = userdata.get('open_weather')

In [47]:
city = "Provo"

url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={token}&units=imperial"

KAFKA_TOPIC = "sample-streaming-data"  # Kafka topic to store data


In [48]:
def error_callback(exc):
    raise Exception('Error while sending data to Kafka: {0}'.format(str(exc)))

In [49]:
def write_to_kafka(topic_name, items):
    producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
    count = 0
    for message, key in items:
        producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8'), partition=0).add_errback(error_callback)
        count += 1
    producer.flush()
    print(f"Wrote {count} messages into topic: {topic_name}")



In [52]:
# Produce weather data every 5 seconds for 5 minutes
def produce_weather_data():
    items = []
    for _ in range(60):  # Fetch data every 5 seconds for 5 minutes
        response = requests.get(url)
        data = response.json()
        current_temp = data['main']['temp']
        timestamp = str(time.time())

        # Add message to the list
        items.append((str(current_temp), timestamp))
        print(f"Produced: {timestamp} - {current_temp}")

        time.sleep(5)

    write_to_kafka(KAFKA_TOPIC, items)

produce_weather_data()

Produced: 1729964252.8536134 - 60.93
Produced: 1729964257.9904437 - 60.93
Produced: 1729964263.022206 - 61.92
Produced: 1729964268.0516603 - 60.93
Produced: 1729964273.0796666 - 61.92
Produced: 1729964278.1049638 - 61.92
Produced: 1729964283.1290257 - 61.92
Produced: 1729964288.1546233 - 61.92
Produced: 1729964293.1817122 - 61.92
Produced: 1729964298.2023797 - 61.92
Produced: 1729964303.2297723 - 60.93
Produced: 1729964308.2550464 - 61.92
Produced: 1729964313.283817 - 60.93
Produced: 1729964318.3043072 - 60.93
Produced: 1729964323.332275 - 60.93
Produced: 1729964328.356748 - 60.93
Produced: 1729964333.3794832 - 60.93
Produced: 1729964338.4037025 - 60.93
Produced: 1729964343.4249218 - 60.93
Produced: 1729964348.456297 - 61.2
Produced: 1729964353.4820545 - 61.2
Produced: 1729964358.5062025 - 61.2
Produced: 1729964363.5320034 - 61.2
Produced: 1729964368.5560515 - 61.2
Produced: 1729964373.5808704 - 61.2
Produced: 1729964378.6088264 - 61.2
Produced: 1729964383.6292365 - 61.2
Produced: 1729

# Task II - Extract data from the Kafka stream as a Consumer.

In [53]:
from kafka import KafkaConsumer

# Kafka consumer configuration
bootstrap_servers = ['127.0.0.1:9092']
group_id = 'weather_consumer'
topic_name = KAFKA_TOPIC  # The same topic created in Task I

# Initialize Kafka consumer
consumer = KafkaConsumer(
    topic_name,
    bootstrap_servers=bootstrap_servers,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id=group_id,
    # Decodes the message value
    value_deserializer=lambda x: x.decode('utf-8'),
    # Decodes the message key
    key_deserializer=lambda x: x.decode('utf-8')
)

# Read messages and store them in a list
def consume_weather_data(message_count=60):
    messages = []
    try:
        for _ in range(message_count):
            message = next(consumer, None)  # Use a default None if no message is found
            if message is None:  # Exit if no more messages are received
                print("No more messages available.")
                break
            timestamp, temperature = message.key, float(message.value)
            print(f"Consumed: {timestamp} - {temperature}")
            messages.append((timestamp, temperature))
    finally:
        consumer.close()
    return messages

# Consume data from Kafka
weather_data = consume_weather_data()

Consumed: 1729964252.8536134 - 60.93
Consumed: 1729964257.9904437 - 60.93
Consumed: 1729964263.022206 - 61.92
Consumed: 1729964268.0516603 - 60.93
Consumed: 1729964273.0796666 - 61.92
Consumed: 1729964278.1049638 - 61.92
Consumed: 1729964283.1290257 - 61.92
Consumed: 1729964288.1546233 - 61.92
Consumed: 1729964293.1817122 - 61.92
Consumed: 1729964298.2023797 - 61.92
Consumed: 1729964303.2297723 - 60.93
Consumed: 1729964308.2550464 - 61.92
Consumed: 1729964313.283817 - 60.93
Consumed: 1729964318.3043072 - 60.93
Consumed: 1729964323.332275 - 60.93
Consumed: 1729964328.356748 - 60.93
Consumed: 1729964333.3794832 - 60.93
Consumed: 1729964338.4037025 - 60.93
Consumed: 1729964343.4249218 - 60.93
Consumed: 1729964348.456297 - 61.2
Consumed: 1729964353.4820545 - 61.2
Consumed: 1729964358.5062025 - 61.2
Consumed: 1729964363.5320034 - 61.2
Consumed: 1729964368.5560515 - 61.2
Consumed: 1729964373.5808704 - 61.2
Consumed: 1729964378.6088264 - 61.2
Consumed: 1729964383.6292365 - 61.2
Consumed: 1729

# Task III - Convert the messages to a format for adding to a sqlite database table

In [54]:
import sqlite3

# Initialize SQLite database connection
conn = sqlite3.connect("weather_data.db")
cursor = conn.cursor()

# Create table if not exists
cursor.execute('''CREATE TABLE IF NOT EXISTS weather (timestamp TEXT, temperature REAL)''')
conn.commit()

# Insert consumed data into SQLite database
def store_data_in_db(data):
    cursor.executemany("INSERT INTO weather (timestamp, temperature) VALUES (?, ?)", data)
    conn.commit()

# Store the consumed data
store_data_in_db(weather_data)

# Viewing the first few rows of the data
pd.read_sql_query("SELECT * FROM weather LIMIT 5", conn)


,timestamp,temperature
0,1729964252.8536134,60.93
1,1729964257.9904437,60.93
2,1729964263.022206,61.92
3,1729964268.0516603,60.93
4,1729964273.0796666,61.92


# Task IV - SQL summarization

In [55]:
def summarize_data():
    cursor.execute("SELECT AVG(temperature) FROM weather")
    avg_temp = cursor.fetchone()[0]
    print(f"Average Temperature of Draper over the last 5 minutes: {avg_temp}°F")

summarize_data()

# Close SQLite connection
conn.close()

Average Temperature of Draper over the last 5 minutes: 61.18349999999994°F


# Task V - Create an Airflow DAG with function

In [58]:
# replace ###### with your file name
# make sure you have your google drive mounted.

!cp "/content/drive/MyDrive/Colab Notebooks/A8_Vu_Nguyen.ipynb" ./
!jupyter nbconvert --to html "A8_Vu_Nguyen.ipynb"

[NbConvertApp] Converting notebook A8_Vu_Nguyen.ipynb to html
[NbConvertApp] Writing 639262 bytes to A8_Vu_Nguyen.html
